In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean,udf
from unidecode import unidecode

In [ ]:
def quitar_acentos_y_L(cadena):
    cadena_sin_acentos = unidecode(cadena)
    return cadena_sin_acentos.replace("L'", "")

In [ ]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

In [ ]:
DF1 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM renda") \
  .load()
DF1.show()

In [ ]:
DF2 = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/freshdata.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT * FROM rent_price") \
  .load()
DF2.show()

In [ ]:
quitar_acentos_y_L_udf = udf(quitar_acentos_y_L, StringType())

# Aplicar la función UDF a la columna 'texto_con_acentos' y crear una nueva columna
DF1 = DF1.withColumn('texto_sin_acentos', quitar_acentos_y_L_udf(DF1['texto_con_acentos']))

# Mostrar el DataFrame resultante
DF1.show()

In [ ]:
DF1.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
        .option("dbtable", "renda") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .mode("overwrite") \
        .save()